# MAIN

In [35]:

%load_ext autoreload
%autoreload 2

import os
import shutil
from pathlib import Path
import pandas as pd 
import logging
from src.data.get_data import get_data_reco_custdim_spdim , get_kyc_customers, get_customer_score
from src.data.get_connection import get_connection
from datetime import datetime, timedelta 
from src.data.preprocessing import preprocessing
from src.routing.ValhallaManager import ValhallaManager
from src.data.data_filter import data_filter 
from src.data.export import export_data 
from src.clustering.evaluate_cluster import evaluate_unsupervised_clustering
from src.utils_and_postprocessing.utils import cluster_summary_and_selection, postprocess_selected_trip
from src.utils_and_postprocessing.run_clustering_and_routing import create_and_plot_route, create_cluster_trip_optroute 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
%reload_ext autoreload

### Logger Set-up

In [37]:
import logging
import os # Good practice to import os for path manipulation, especially for log files

# Get the logger instance for the current module
logger = logging.getLogger(__name__)

# Set the overall logging level for the logger.
# Messages below this level will be ignored by this logger.
logger.setLevel(logging.INFO)

# --- 1. File Handler (for logging to a file) ---
log_file_path = 'test.log' # You might want to make this dynamic later
file_handler = logging.FileHandler(log_file_path)
file_handler.setLevel(logging.INFO) # Set the level for this handler
file_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(file_formatter)
logger.addHandler(file_handler)

# --- 2. Stream Handler (for logging to the console/notebook output) ---
stream_handler = logging.StreamHandler() # By default, this logs to sys.stderr
stream_handler.setLevel(logging.INFO) # Set the level for this handler
stream_formatter = logging.Formatter('%(levelname)s: %(message)s') # Often a simpler format for console
stream_handler.setFormatter(stream_formatter)
logger.addHandler(stream_handler)


### Parameter Setup

In [38]:
# Constants for directory structure
# BASE_DIR = Path(__file__).resolve().parent
BASE_DIR = Path('').resolve()#.parent
RECOMMENDATION_DIR = BASE_DIR / 'recommendation_output'
SELECTED_TRIP_DIR = 'selected_trip_map'
CLUSTER_MAP_DIR = 'cluster_map'
EXCEL_DOCS_DIR = 'excel_docs'
INDEX_HTML = 'index.html'

# Global variables
CURRENT_DATE = None
SELECTED_TRIP_PATH = None
ALL_CLUSTER_PATH = None
LOCAL_EXCEL_PATH = None

def setup_directories():
    """Set up directory structure for recommendation output with date-based subdirectories."""
    global CURRENT_DATE, SELECTED_TRIP_PATH, ALL_CLUSTER_PATH, LOCAL_EXCEL_PATH
    try:
        # Set current date
        CURRENT_DATE = datetime.today().date()
        
        # Define date-based directory paths
        SELECTED_TRIP_PATH = RECOMMENDATION_DIR / SELECTED_TRIP_DIR / str(CURRENT_DATE)
        ALL_CLUSTER_PATH = RECOMMENDATION_DIR / CLUSTER_MAP_DIR / str(CURRENT_DATE)
        LOCAL_EXCEL_PATH = RECOMMENDATION_DIR / EXCEL_DOCS_DIR / str(CURRENT_DATE)
        
        # Create directories if they don't exist
        for directory in (SELECTED_TRIP_PATH, ALL_CLUSTER_PATH, LOCAL_EXCEL_PATH):
            directory.mkdir(parents=True, exist_ok=True)
            logger.info(f"Ensured directory exists: {directory}")

        # Copy index.html files from source directories to date-based subdirectories
        source_cluster_map_index = BASE_DIR / 'html' / 'default_cluster_map_index.html'
        source_selected_trip_index = BASE_DIR / 'html' / 'default_selected_cluster_map_index.html' 

        # Copy index.html for selected_trip_map
        try:
            if source_selected_trip_index.exists():
                shutil.copy2(source_selected_trip_index, SELECTED_TRIP_PATH / INDEX_HTML)
                logger.info(f"Copied index.html to {SELECTED_TRIP_PATH / INDEX_HTML}")
            else:
                logger.warning(f"Source index.html not found at {source_selected_trip_index}")
        except Exception as e:
            logger.error(f"Failed to copy index.html to {SELECTED_TRIP_PATH}: {str(e)}")

        # Copy index.html for cluster_map
        try:
            if source_cluster_map_index.exists():
                shutil.copy2(source_cluster_map_index, ALL_CLUSTER_PATH / INDEX_HTML)
                logger.info(f"Copied index.html to {ALL_CLUSTER_PATH / INDEX_HTML}")
            else:
                logger.warning(f"Source index.html not found at {source_cluster_map_index}")
        except Exception as e:
            logger.error(f"Failed to copy index.html to {ALL_CLUSTER_PATH}: {str(e)}")

    except Exception as e:
        logger.error(f"Error setting up directories: {str(e)}")
        raise

if __name__ == "__main__":
    # Set up basic logging configuration
    logging.basicConfig(level=logging.INFO)
    setup_directories()

INFO: Ensured directory exists: /home/azureuser/BT/11_Demand_Engine/Algorithm_V1/recommendation_output/selected_trip_map/2025-07-01
INFO: Ensured directory exists: /home/azureuser/BT/11_Demand_Engine/Algorithm_V1/recommendation_output/selected_trip_map/2025-07-01
INFO: Ensured directory exists: /home/azureuser/BT/11_Demand_Engine/Algorithm_V1/recommendation_output/selected_trip_map/2025-07-01
INFO:__main__:Ensured directory exists: /home/azureuser/BT/11_Demand_Engine/Algorithm_V1/recommendation_output/selected_trip_map/2025-07-01
INFO: Ensured directory exists: /home/azureuser/BT/11_Demand_Engine/Algorithm_V1/recommendation_output/cluster_map/2025-07-01
INFO: Ensured directory exists: /home/azureuser/BT/11_Demand_Engine/Algorithm_V1/recommendation_output/cluster_map/2025-07-01
INFO: Ensured directory exists: /home/azureuser/BT/11_Demand_Engine/Algorithm_V1/recommendation_output/cluster_map/2025-07-01
INFO:__main__:Ensured directory exists: /home/azureuser/BT/11_Demand_Engine/Algorithm_

In [39]:
# Instantiate the manager
valhalla_manager = ValhallaManager(logger=logger)

# Start the server
valhalla_manager.start_valhalla()

# Check valhalla status
valhalla_manager.check_valhalla_status()

Starting Valhalla container...
 Network valhalla_nigeria_project_default  Creating
 Network valhalla_nigeria_project_default  Created
 Container valhalla_nigeria_project-valhalla-1  Creating
 Container valhalla_nigeria_project-valhalla-1  Created
 Container valhalla_nigeria_project-valhalla-1  Starting
 Container valhalla_nigeria_project-valhalla-1  Started

Waiting 10 seconds for Valhalla to initialize...
Valhalla container started.


True

## Get Data

In [40]:
def get_input_data(logger=logging.getLogger(__name__)):
    # Get the data recommendation, customer dimension with affinity score, and stockpoint dimension
    df_customer_sku_recommendation_raw,  df_customer_dim_with_affinity_score_raw, df_stockpoint_dim_raw =   get_data_reco_custdim_spdim(logger=logger) # 1mins # 3mins  
    
    # Get KYC customers ------------------------
    df_kyc_customer = get_kyc_customers(logger=logger) 
    logger.info(f"KYC customers DataFrame shape: {df_kyc_customer.shape}")
    
    # Get customer scores ---------------------------------------------
    df_customer_score = get_customer_score(logger=logger) # ETA: 40 mins
    
    df_customer_days_since_last_order = df_customer_score.groupby('CustomerID').days_since_last_order.min().reset_index()
    # Update df_customer_score with df_customer_days_since_last_order
    df_customer_score.drop(columns=['days_since_last_order'], inplace=True, errors='ignore')
    df_customer_score = df_customer_score.merge(df_customer_days_since_last_order, on='CustomerID', suffixes=('', '_min'))
    logger.info(f"Customer scores DataFrame shape: {df_customer_score.shape}")
     
    
    return df_customer_sku_recommendation_raw, df_customer_dim_with_affinity_score_raw, df_stockpoint_dim_raw, df_kyc_customer, df_customer_score

## Main Function

In [41]:
def run_push_recommendation(df_customer_sku_recommendation, 
                            df_master_customer_dim, 
                            df_stockpoint_dim, 
                            stock_point_id,
                            stock_point_name,
                            sku_recency = 7, 
                            customer_recency = 60, number_recommendation = 5, 
                            estimate_qty_scale_factor = 1, max_estimated_qty = 5, 
                            exclude_recency_customer = 4,
                            max_customers_per_route=20,
                            max_volume_per_route=300,
                            max_distance_km = 40,
                            sel_trip_cluster = 5,
                            min_ncust_per_cluster = 5,
                            clustering_method = 'divisive',
                            skip_route_optimization = False,
                            save_to_disk = False,
                            logger=logging.getLogger(__name__)):
    """
    Main execution function demonstrating complete route optimization workflow
    """ 

    logger.info("=" * 80)
    logger.info("ROUTE OPTIMIZATION FOR PUSH SALES RECOMMENDATIONS")
    logger.info(f"StockPoint: {stock_point_name}, StockPointID: {stock_point_id},")
    logger.info("=" * 80)

    # STEP 1: Load or Generate Data
    logger.info("\n1. Loading Data...")
    logger.info("-" * 40)

    df_sku_rec, df_customer_dim, df_stockpoint  = data_filter(df_customer_sku_recommendation, 
                                                                df_master_customer_dim, 
                                                                df_stockpoint_dim, 
                                                                stockpoint_id = stock_point_id,  
                                                                sku_recency = sku_recency, 
                                                                customer_recency = customer_recency, 
                                                                number_recommendation = number_recommendation,
                                                                estimate_qty_scale_factor = estimate_qty_scale_factor, 
                                                                max_estimated_qty = max_estimated_qty,
                                                                exclude_recency_customer = exclude_recency_customer)

    # if len(df_customer_dim) < min_ncust_per_cluster:
        # return {}

    logger.info(f"✓ Loaded {len(df_sku_rec)} SKU recommendations")
    logger.info(f"✓ Loaded {len(df_customer_dim)} customer records")
    logger.info(f"✓ Loaded {len(df_stockpoint)} stock points")

    push_recommendation, df_clustering, df_routes, stock_point_coords = create_cluster_trip_optroute(df_sku_rec, 
                                                                                            df_customer_dim, 
                                                                                            df_stockpoint,
                                                                                            stock_point_id,
                                                                                            max_customers_per_route, 
                                                                                            max_volume_per_route,
                                                                                            max_distance_km,
                                                                                            clustering_method,
                                                                                            skip_route_optimization,
                                                                                            logger=logger)


    ### Cluster Summary 
    cluster_summary, df_high_value_cluster_summary, sel_cluster_tuple, sel_total_customer_count = cluster_summary_and_selection(
                                                                                                        push_recommendation,
                                                                                                        sel_trip_cluster,
                                                                                                        min_ncust_per_cluster = min_ncust_per_cluster,
                                                                                                        logger=logger)

    ## Trip
    selected_push_recommendation_trip, all_push_recommendation = postprocess_selected_trip(push_recommendation, 
                                                    cluster_summary, 
                                                    df_master_customer_dim,  
                                                    df_stockpoint,
                                                    sel_cluster_tuple)


    ## Cluster Evaluation
    try:
        evaluate_unsupervised_clustering(selected_push_recommendation_trip)
    except:
        pass
    
    ## Selected Trip Summary
    df_selected_trip_summary =  (selected_push_recommendation_trip
                                 .groupby(['StockPointID','TripID','CustomerID', 'Latitude','Longitude',
                                           'LGA', 'LCDA','CustomerScore'])
                                 .agg( TotalQuantity = ('EstimatedQuantity','sum')
                                      ,TotalSKU = ('SKUID','nunique'))
                                 .reset_index()
                                )
    
    ## Create selected trip route and map
    try:
        create_and_plot_route(df_cluster_or_trip = df_selected_trip_summary, 
                              df_stockpoint_dim = df_stockpoint_dim, 
                              main_dir = SELECTED_TRIP_PATH, 
                              valhalla_manager = valhalla_manager, 
                              calc_route = True, 
                              logger = logger)
    except Exception as e:
        logger.error(f"Error in creating and plotting route: {e}")
    
    ## Create and save cluster map
    try:
        create_and_plot_route(df_cluster_or_trip = df_clustering, 
                      df_stockpoint_dim = df_stockpoint_dim, 
                      main_dir=ALL_CLUSTER_PATH, 
                          valhalla_manager=None, calc_route = False, 
                          logger=logger, cluster_col='cluster')
    except Exception as e:
        logger.info(f'Unable to generate and save cluster map: {e}') 
    
    ## Local Export
    ### Export Data
    if save_to_disk:
        try:
            export_data(
                selected_trip = selected_push_recommendation_trip,
                all_push_recommendation = all_push_recommendation,
                cluster_summary = cluster_summary,
                stock_point_name = stock_point_name,
                CURRENT_DATE = CURRENT_DATE,
                LOCAL_EXCEL_PATH  = LOCAL_EXCEL_PATH
            )
        except Exception as e:
            logger.error(f'Can not save to disk {e}')
     

    dict_ = {
        'stock_point_name': stock_point_name,
        'selected_trip': selected_push_recommendation_trip,
        'all_push_recommendation': all_push_recommendation,
        'cluster_summary': cluster_summary
    }

    return dict_
    
    

## MAIN 1

In [42]:
# Get input data
df_customer_sku_recommendation_raw, df_customer_dim_with_affinity_score_raw, \
    df_stockpoint_dim_raw, df_kyc_customer, df_customer_score = get_input_data(logger=logging.getLogger(__name__))

INFO: Executing stored procedure(s) to fetch data...
INFO: Executing stored procedure(s) to fetch data...
INFO: Executing stored procedure(s) to fetch data...
INFO:__main__:Executing stored procedure(s) to fetch data...
INFO: Data fetch complete.
INFO: Data fetch complete.
INFO: Data fetch complete.
INFO:__main__:Data fetch complete.
INFO: --- Fetched DataFrames Shapes ---
INFO: --- Fetched DataFrames Shapes ---
INFO: --- Fetched DataFrames Shapes ---
INFO:__main__:--- Fetched DataFrames Shapes ---
INFO: Customer SKU Recommendation: (155758, 13)
INFO: Customer SKU Recommendation: (155758, 13)
INFO: Customer SKU Recommendation: (155758, 13)
INFO:__main__:Customer SKU Recommendation: (155758, 13)
INFO: Customer Dimension with Affinity Score: (10793, 22)
INFO: Customer Dimension with Affinity Score: (10793, 22)
INFO: Customer Dimension with Affinity Score: (10793, 22)
INFO:__main__:Customer Dimension with Affinity Score: (10793, 22)
INFO: Stockpoint Dimension: (72, 4)
INFO: Stockpoint Dim

In [43]:
# Preprocessing
df_customer_sku_recommendation, df_master_customer_dim, df_stockpoint_dim = preprocessing(df_customer_sku_recommendation_raw, 
                                                                                            df_customer_dim_with_affinity_score_raw, 
                                                                                            df_stockpoint_dim_raw,
                                                                                            df_customer_score,
                                                                                            df_kyc_customer)

In [44]:
# Data Filter - Testing 
df_sku_rec, df_customer_dim, df_stockpoint = data_filter(df_customer_sku_recommendation, 
                                                                    df_master_customer_dim, 
                                                                    df_stockpoint_dim, 
                                                                    stockpoint_id = 1647394,  
                                                                    # stockpoint_id = 1647113,  
                                                                    sku_recency = 7, customer_recency = 60, number_recommendation = 10,
                                                                    estimate_qty_scale_factor = 1, max_estimated_qty = 5, 
                                                                    exclude_recency_customer = 4)

Total Quantity before filter: 16,192
Total Quantity: 11,991
Total Number of Customers before filter: 476
Total Number of Customers: 353


In [ ]:
# stock_point_id = 1647394 #
stock_point_id =  1647113
stock_point_name = df_stockpoint_dim.query(f'Stock_Point_ID == {1647394}')['Stock_point_Name'].iloc[0] 
res_dict = run_push_recommendation(df_customer_sku_recommendation, 
                            df_master_customer_dim, 
                            df_stockpoint_dim, 
                            stock_point_id,
                            stock_point_name,
                            sku_recency = 7, 
                            customer_recency = 60, number_recommendation = 5, 
                            estimate_qty_scale_factor = 1, max_estimated_qty = 5, 
                            exclude_recency_customer = 4,
                            max_customers_per_route=20,
                            max_volume_per_route=300,
                            max_distance_km = 40,
                            sel_trip_cluster = 5,
                            min_ncust_per_cluster = 5,
                            clustering_method = 'divisive',
                            skip_route_optimization = False,
                            save_to_disk = False,
                            logger=logger)

In [52]:
ALL_STOCKPOINTS_RESULT = {}
for index, row in df_stockpoint_dim.iterrows():
    # if index == 12:
    # if index == 5:
    stock_point_id =  row['Stock_Point_ID']
    stock_point_name = row['Stock_point_Name']
    print(f'{index}/{len(df_stockpoint_dim)} \nStock Point ID: {stock_point_id} || Stock Point Name: {stock_point_name}')  # Access by column name

    res_dict = run_push_recommendation(df_customer_sku_recommendation, 
                            df_master_customer_dim, 
                            df_stockpoint_dim, 
                            stock_point_id,
                            stock_point_name,
                            sku_recency = 7, 
                            customer_recency = 60, number_recommendation = 5, 
                            estimate_qty_scale_factor = 1, max_estimated_qty = 5, 
                            exclude_recency_customer = 4,
                            max_customers_per_route=20,
                            max_volume_per_route=300,
                            max_distance_km = 40,
                            sel_trip_cluster = 5,
                            min_ncust_per_cluster = 5,
                            clustering_method = 'divisive',
                            skip_route_optimization = False,
                            save_to_disk = False,
                            logger=logger)
    
    ALL_STOCKPOINTS_RESULT[stock_point_name] = res_dict

INFO: ================================================================================
INFO: ================================================================================
INFO: ================================================================================
INFO:__main__:================================================================================
INFO: ROUTE OPTIMIZATION FOR PUSH SALES RECOMMENDATIONS
INFO: ROUTE OPTIMIZATION FOR PUSH SALES RECOMMENDATIONS
INFO: ROUTE OPTIMIZATION FOR PUSH SALES RECOMMENDATIONS
INFO:__main__:ROUTE OPTIMIZATION FOR PUSH SALES RECOMMENDATIONS
INFO: StockPoint: OmniHub Obio Akpor Rivers - Rivoc, StockPointID: 1647128,
INFO: StockPoint: OmniHub Obio Akpor Rivers - Rivoc, StockPointID: 1647128,
INFO: StockPoint: OmniHub Obio Akpor Rivers - Rivoc, StockPointID: 1647128,
INFO:__main__:StockPoint: OmniHub Obio Akpor Rivers - Rivoc, StockPointID: 1647128,
INFO: ================================================================================
INFO: =======

0/72 
Stock Point ID: 1647128 || Stock Point Name: OmniHub Obio Akpor Rivers - Rivoc
Total Quantity before filter: 25
Total Quantity: 0
Total Number of Customers before filter: 1
Total Number of Customers: 0
1/72 
Stock Point ID: 1647401 || Stock Point Name: OmniHub Ado Odo/Ota Ogun - Prince Tunadek


INFO: ✓ Loaded 11 SKU recommendations
INFO: ✓ Loaded 11 SKU recommendations
INFO: ✓ Loaded 11 SKU recommendations
INFO:__main__:✓ Loaded 11 SKU recommendations
INFO: ✓ Loaded 4 customer records
INFO: ✓ Loaded 4 customer records
INFO: ✓ Loaded 4 customer records
INFO:__main__:✓ Loaded 4 customer records
INFO: ✓ Loaded 1 stock points
INFO: ✓ Loaded 1 stock points
INFO: ✓ Loaded 1 stock points
INFO:__main__:✓ Loaded 1 stock points
INFO: ✓ Route optimizer initialized
INFO: ✓ Route optimizer initialized
INFO: ✓ Route optimizer initialized
INFO:__main__:✓ Route optimizer initialized
INFO: 
3. Generating Optimized Routes...
INFO: 
3. Generating Optimized Routes...
INFO: 
3. Generating Optimized Routes...
INFO:__main__:
3. Generating Optimized Routes...
INFO: ----------------------------------------
INFO: ----------------------------------------
INFO: ----------------------------------------
INFO:__main__:----------------------------------------
INFO: ✓ Created 1 clusters for Stock Point 16474

Total Quantity before filter: 193
Total Quantity: 49
Total Number of Customers before filter: 19
Total Number of Customers: 4
2/72 
Stock Point ID: 1647402 || Stock Point Name: OmniHub AMAC 1 Abuja - Elriah


INFO: ✓ Loaded 15 SKU recommendations
INFO: ✓ Loaded 15 SKU recommendations
INFO: ✓ Loaded 15 SKU recommendations
INFO:__main__:✓ Loaded 15 SKU recommendations
INFO: ✓ Loaded 5 customer records
INFO: ✓ Loaded 5 customer records
INFO: ✓ Loaded 5 customer records
INFO:__main__:✓ Loaded 5 customer records
INFO: ✓ Loaded 1 stock points
INFO: ✓ Loaded 1 stock points
INFO: ✓ Loaded 1 stock points
INFO:__main__:✓ Loaded 1 stock points
INFO: ✓ Route optimizer initialized
INFO: ✓ Route optimizer initialized
INFO: ✓ Route optimizer initialized
INFO:__main__:✓ Route optimizer initialized
INFO: 
3. Generating Optimized Routes...
INFO: 
3. Generating Optimized Routes...
INFO: 
3. Generating Optimized Routes...
INFO:__main__:
3. Generating Optimized Routes...
INFO: ----------------------------------------
INFO: ----------------------------------------
INFO: ----------------------------------------
INFO:__main__:----------------------------------------
INFO: ✓ Created 1 clusters for Stock Point 16474

...

INFO: 4. Route Analysis & Results...
INFO: 4. Route Analysis & Results...
INFO:__main__:4. Route Analysis & Results...
INFO: ----------------------------------------
INFO: ----------------------------------------
INFO: ----------------------------------------
INFO:__main__:----------------------------------------
INFO: Select ClusterIDs: []
INFO: Select ClusterIDs: []
INFO: Select ClusterIDs: []
INFO:__main__:Select ClusterIDs: []
INFO: Total Number of Customers: 0
INFO: Total Number of Customers: 0
INFO: Total Number of Customers: 0
INFO:__main__:Total Number of Customers: 0
INFO: Empty DataFrame
Columns: [cluster, LGA_list, LCDA_List, ncustomer, totalQty, avg_customer_score]
Index: []
INFO: Empty DataFrame
Columns: [cluster, LGA_list, LCDA_List, ncustomer, totalQty, avg_customer_score]
Index: []
INFO: Empty DataFrame
Columns: [cluster, LGA_list, LCDA_List, ncustomer, totalQty, avg_customer_score]
Index: []
INFO:__main__:Empty DataFrame
Columns: [cluster, LGA_list, LCDA_List, ncustome

69/72 
Stock Point ID: 1647436 || Stock Point Name: OmniHub Egor Edo - 10TH Gear
Total Quantity before filter: 0
Total Quantity: 0
Total Number of Customers before filter: 0
Total Number of Customers: 0
70/72 
Stock Point ID: 1647437 || Stock Point Name: OmniHub Ado Odo/Ota Ogun - Azacram


INFO: ✓ Loaded 0 SKU recommendations
INFO: ✓ Loaded 0 SKU recommendations
INFO: ✓ Loaded 0 SKU recommendations
INFO:__main__:✓ Loaded 0 SKU recommendations
INFO: ✓ Loaded 0 customer records
INFO: ✓ Loaded 0 customer records
INFO: ✓ Loaded 0 customer records
INFO:__main__:✓ Loaded 0 customer records
INFO: ✓ Loaded 1 stock points
INFO: ✓ Loaded 1 stock points
INFO: ✓ Loaded 1 stock points
INFO:__main__:✓ Loaded 1 stock points
INFO: ✓ Route optimizer initialized
INFO: ✓ Route optimizer initialized
INFO: ✓ Route optimizer initialized
INFO:__main__:✓ Route optimizer initialized
INFO: 
3. Generating Optimized Routes...
INFO: 
3. Generating Optimized Routes...
INFO: 
3. Generating Optimized Routes...
INFO:__main__:
3. Generating Optimized Routes...
INFO: ----------------------------------------
INFO: ----------------------------------------
INFO: ----------------------------------------
INFO:__main__:----------------------------------------
INFO: ✓ Created 0 clusters for Stock Point 1647437 u

Total Quantity before filter: 0
Total Quantity: 0
Total Number of Customers before filter: 0
Total Number of Customers: 0
71/72 
Stock Point ID: 1647438 || Stock Point Name: OmniHub Ado Odo/Ota Ogun - Soffy's Concept
Total Quantity before filter: 0
Total Quantity: 0
Total Number of Customers before filter: 0
Total Number of Customers: 0


INFO: 4. Route Analysis & Results...
INFO: 4. Route Analysis & Results...
INFO: 4. Route Analysis & Results...
INFO:__main__:4. Route Analysis & Results...
INFO: ----------------------------------------
INFO: ----------------------------------------
INFO: ----------------------------------------
INFO:__main__:----------------------------------------
INFO: Select ClusterIDs: []
INFO: Select ClusterIDs: []
INFO: Select ClusterIDs: []
INFO:__main__:Select ClusterIDs: []
INFO: Total Number of Customers: 0
INFO: Total Number of Customers: 0
INFO: Total Number of Customers: 0
INFO:__main__:Total Number of Customers: 0
INFO: Empty DataFrame
Columns: [cluster, LGA_list, LCDA_List, ncustomer, totalQty, avg_customer_score]
Index: []
INFO: Empty DataFrame
Columns: [cluster, LGA_list, LCDA_List, ncustomer, totalQty, avg_customer_score]
Index: []
INFO: Empty DataFrame
Columns: [cluster, LGA_list, LCDA_List, ncustomer, totalQty, avg_customer_score]
Index: []
INFO:__main__:Empty DataFrame
Columns: [c

## Export to DB

In [ ]:
from src.data.export2db import RecommendationProcessor
from src.data.get_connection import get_connection

In [ ]:
# Simple usage
# main(ALL_STOCKPOINTS_RESULT, CURRENT_DATE, get_connection)

# Or use the processor directly
processor = RecommendationProcessor(get_connection)
processor.process(ALL_STOCKPOINTS_RESULT, CURRENT_DATE)

# Clean Up

In [53]:
# Stop the server when done
valhalla_manager.stop_valhalla()

Stopping Valhalla container...
 Container valhalla_nigeria_project-valhalla-1  Stopping
 Container valhalla_nigeria_project-valhalla-1  Stopped
 Container valhalla_nigeria_project-valhalla-1  Removing
 Container valhalla_nigeria_project-valhalla-1  Removed
 Network valhalla_nigeria_project_default  Removing
 Network valhalla_nigeria_project_default  Removed

Valhalla container stopped.
